In [1]:
import numpy as np
import pandas as pd
import re
from gensim.models import Word2Vec
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense
from keras.models import Sequential
from sklearn.metrics import classification_report

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2015train-A.txt',
    './dataset/train/twitter-2016train-A.txt',
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

mode = 'N-R'  # 可以選擇 'N-R' 或 'R'

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練 Word2Vec 模型
tokenized_sentences = [tweet.split() for tweet in dataset['tweet'].values]
word2vec_model = Word2Vec(sentences=tokenized_sentences,
                          vector_size=EMBEDDING_DIM,  # 詞向量維度為 25
                          window=5,                   # 最大跳躍距離設為 5
                          min_count=5,                # 最低頻次為 5
                          sg=0,                       # CBOW 模型 (sg=0)
                          workers=4)                  # 使用 4 個核心進行訓練

# 將詞向量進行標準化
def normalize_vector(vector):
    v_min, v_max = np.min(vector), np.max(vector)
    return (vector - v_min) / (v_max - v_min) if v_max > v_min else vector

# 構建嵌入矩陣
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    if word in word2vec_model.wv:
        embedding_vector = word2vec_model.wv[word]
        embedding_matrix[i] = normalize_vector(embedding_vector)

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

### 建立單一 CNN 模型並加入 Flatten 層

In [2]:
from keras.layers import Flatten

cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 2s 4ms/step - loss: 1.0102 - accuracy: 0.4521 - val_loss: 0.9915 - val_accuracy: 0.4702
Epoch 2/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9927 - accuracy: 0.4784 - val_loss: 0.9798 - val_accuracy: 0.4751
Epoch 3/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9851 - accuracy: 0.4823 - val_loss: 0.9784 - val_accuracy: 0.4708
Epoch 4/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9812 - accuracy: 0.4911 - val_loss: 0.9822 - val_accuracy: 0.4652
Epoch 5/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9773 - accuracy: 0.4934 - val_loss: 0.9689 - val_accuracy: 0.4943
Epoch 6/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9726 - accuracy: 0.4983 - val_loss: 0.9676 - val_accuracy: 0.4903
Epoch 7/10
405/405 [==============================] - 1s 3ms/step - loss: 0.9708 - accuracy: 0.4995 - val_loss: 0.9657 - val_accuracy: 0.4940
Epoch 

### 單一 LSTM 模型 (Single LSTM)

In [ ]:
from keras.layers import LSTM

lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 31s 68ms/step - loss: 1.0027 - accuracy: 0.4590 - val_loss: 0.9820 - val_accuracy: 0.4628
Epoch 2/10
405/405 [==============================] - 28s 70ms/step - loss: 0.9871 - accuracy: 0.4797 - val_loss: 0.9736 - val_accuracy: 0.4739
Epoch 3/10
405/405 [==============================] - 30s 73ms/step - loss: 0.9806 - accuracy: 0.4825 - val_loss: 0.9690 - val_accuracy: 0.4813
Epoch 4/10
405/405 [==============================] - 36s 88ms/step - loss: 0.9761 - accuracy: 0.4879 - val_loss: 0.9594 - val_accuracy: 0.4927
Epoch 5/10
405/405 [==============================] - 55s 135ms/step - loss: 0.9746 - accuracy: 0.4928 - val_loss: 0.9631 - val_accuracy: 0.4909
Epoch 6/10
405/405 [==============================] - 28s 69ms/step - loss: 0.9731 - accuracy: 0.4906 - val_loss: 0.9557 - val_accuracy: 0.5100
Epoch 7/10
405/405 [==============================] - 27s 67ms/step - loss: 0.9706 - accuracy: 0.4991 - val_loss: 0.9540 - val_accuracy

### CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

In [4]:
# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 16s 31ms/step - loss: 1.0062 - accuracy: 0.4535 - val_loss: 0.9863 - val_accuracy: 0.4810
Epoch 2/10
405/405 [==============================] - 12s 29ms/step - loss: 0.9898 - accuracy: 0.4755 - val_loss: 0.9679 - val_accuracy: 0.4909
Epoch 3/10
405/405 [==============================] - 13s 31ms/step - loss: 0.9832 - accuracy: 0.4852 - val_loss: 0.9681 - val_accuracy: 0.4878
Epoch 4/10
405/405 [==============================] - 31s 76ms/step - loss: 0.9785 - accuracy: 0.4859 - val_loss: 0.9638 - val_accuracy: 0.4995
Epoch 5/10
405/405 [==============================] - 121s 298ms/step - loss: 0.9747 - accuracy: 0.4906 - val_loss: 0.9673 - val_accuracy: 0.4921
Epoch 6/10
405/405 [==============================] - 119s 294ms/step - loss: 0.9713 - accuracy: 0.4969 - val_loss: 0.9647 - val_accuracy: 0.4912
Epoch 7/10
405/405 [==============================] - 9s 22ms/step - loss: 0.9704 - accuracy: 0.4981 - val_loss: 0.9598 - val_accura

### CNN + LSTM 組合模型 + SVM 組合模型

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

102/102 [==============================] - 81s 794ms/step
              precision    recall  f1-score   support

           0       0.49      0.58      0.53      1372
           1       1.00      0.00      0.00       466
           2       0.52      0.59      0.55      1397

    accuracy                           0.50      3235
   macro avg       0.67      0.39      0.36      3235
weighted avg       0.58      0.50      0.47      3235



### 單一 3 層 CNN 和 LSTM 網絡

In [6]:
three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
237/405 [================>.............] - ETA: 2:04 - loss: 1.0118 - accuracy: 0.4367

405/405 [==============================] - 195s 455ms/step - loss: 1.0089 - accuracy: 0.4382 - val_loss: 0.9930 - val_accuracy: 0.4377
Epoch 2/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9898 - accuracy: 0.4706 - val_loss: 1.0014 - val_accuracy: 0.4476
Epoch 3/10
405/405 [==============================] - 7s 17ms/step - loss: 0.9797 - accuracy: 0.4732 - val_loss: 0.9677 - val_accuracy: 0.4859
Epoch 4/10
405/405 [==============================] - 7s 17ms/step - loss: 0.9726 - accuracy: 0.4872 - val_loss: 0.9672 - val_accuracy: 0.4804
Epoch 5/10
405/405 [==============================] - 7s 17ms/step - loss: 0.9693 - accuracy: 0.4944 - val_loss: 0.9662 - val_accuracy: 0.4807
Epoch 6/10
405/405 [==============================] - 8s 20ms/step - loss: 0.9623 - accuracy: 0.4953 - val_loss: 0.9699 - val_accuracy: 0.4878
Epoch 7/10
405/405 [==============================] - 7s 17ms/step - loss: 0.9596 - accuracy: 0.4990 - val_loss: 0.9672 - val_accuracy: 0.4949
Epoch 8

### 多重 CNN 和 LSTM 網絡

In [7]:
multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 14s 26ms/step - loss: 1.0050 - accuracy: 0.4499 - val_loss: 0.9892 - val_accuracy: 0.4569
Epoch 2/10
405/405 [==============================] - 150s 370ms/step - loss: 0.9878 - accuracy: 0.4719 - val_loss: 0.9697 - val_accuracy: 0.4838
Epoch 3/10
405/405 [==============================] - 10s 26ms/step - loss: 0.9801 - accuracy: 0.4811 - val_loss: 0.9933 - val_accuracy: 0.4556
Epoch 4/10
405/405 [==============================] - 12s 29ms/step - loss: 0.9724 - accuracy: 0.4862 - val_loss: 0.9681 - val_accuracy: 0.4788
Epoch 5/10
405/405 [==============================] - 11s 28ms/step - loss: 0.9687 - accuracy: 0.4993 - val_loss: 0.9583 - val_accuracy: 0.5048
Epoch 6/10
405/405 [==============================] - 12s 30ms/step - loss: 0.9629 - accuracy: 0.5029 - val_loss: 0.9716 - val_accuracy: 0.4788
Epoch 7/10
405/405 [==============================] - 11s 28ms/step - loss: 0.9599 - accuracy: 0.5098 - val_loss: 0.9570 - val_accurac

### 單一 3 層 CNN 和雙向 LSTM 網絡

In [8]:
from keras.layers import Bidirectional

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 17s 25ms/step - loss: 1.0108 - accuracy: 0.4368 - val_loss: 0.9992 - val_accuracy: 0.4529
Epoch 2/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9976 - accuracy: 0.4662 - val_loss: 0.9806 - val_accuracy: 0.4665
Epoch 3/10
405/405 [==============================] - 9s 23ms/step - loss: 0.9826 - accuracy: 0.4794 - val_loss: 0.9745 - val_accuracy: 0.4717
Epoch 4/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9754 - accuracy: 0.4937 - val_loss: 0.9738 - val_accuracy: 0.4893
Epoch 5/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9685 - accuracy: 0.4952 - val_loss: 0.9688 - val_accuracy: 0.4816
Epoch 6/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9669 - accuracy: 0.4978 - val_loss: 0.9614 - val_accuracy: 0.4921
Epoch 7/10
405/405 [==============================] - 10s 24ms/step - loss: 0.9598 - accuracy: 0.5083 - val_loss: 0.9666 - val_accuracy: 0

### 多重 CNN 和雙向 LSTM 網絡

In [9]:
multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 25s 45ms/step - loss: 1.0057 - accuracy: 0.4503 - val_loss: 1.0117 - val_accuracy: 0.4266
Epoch 2/10
405/405 [==============================] - 17s 43ms/step - loss: 0.9876 - accuracy: 0.4708 - val_loss: 0.9677 - val_accuracy: 0.4767
Epoch 3/10
405/405 [==============================] - 18s 44ms/step - loss: 0.9807 - accuracy: 0.4794 - val_loss: 0.9648 - val_accuracy: 0.4804
Epoch 4/10
405/405 [==============================] - 18s 45ms/step - loss: 0.9738 - accuracy: 0.4840 - val_loss: 0.9758 - val_accuracy: 0.4825
Epoch 5/10
405/405 [==============================] - 19s 46ms/step - loss: 0.9701 - accuracy: 0.4974 - val_loss: 0.9659 - val_accuracy: 0.4866
Epoch 6/10
405/405 [==============================] - 20s 49ms/step - loss: 0.9655 - accuracy: 0.4969 - val_loss: 0.9657 - val_accuracy: 0.4890
Epoch 7/10
405/405 [==============================] - 18s 45ms/step - loss: 0.9617 - accuracy: 0.4981 - val_loss: 0.9587 - val_accuracy: